In [49]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import polars as pl

load_dotenv()
engine = create_engine(os.environ["DATABASE_URL"])

with engine.connect() as conn:
    n_orders = conn.execute(text("SELECT COUNT(*) FROM olist.orders")).scalar()
    print("Orders in DB:", n_orders)

    date = conn.execute(text("SELECT shipping_limit_date FROM olist.order_items")).scalar()
    print("Sample shipping limit date:", date)

    df = pl.read_database("SELECT * FROM olist.order_items_expanded", engine)
    display(df)

    first = df.select(pl.col("order_purchase_timestamp").min()).item()
    last = df.select(pl.col("order_purchase_timestamp").max()).item()
    duration = last - first
    print(f"Order purchase timestamps range from {first} to {last} ({duration.days} days)")

    price_sum = df.select(pl.col("price").sum()).item()
    freight_sum = df.select(pl.col("freight_value").sum()).item()
    print(f"Total price of all order items: {price_sum:,.0f}")
    print(f"Total freight value of all order items: {freight_sum:,.0f}")
    print(f"Total revenue (price + freight): {price_sum + freight_sum:,.0f}")


Orders in DB: 99441
Sample shipping limit date: 2017-09-19 09:45:35


order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_delivered_customer_date,customer_id
str,i64,str,str,datetime[μs],"decimal[38,2]","decimal[38,2]",str,datetime[μs],datetime[μs],str
"""00010242fe8c5a6d1ba2dd792cb162…",1,"""4244733e06e7ecb4970a6e2683c13e…","""48436dade18ac8b2bce089ec2a0412…",2017-09-19 09:45:35,58.90,13.29,"""delivered""",2017-09-13 08:59:02,2017-09-20 23:43:48,"""3ce436f183e68e07877b285a838db1…"
"""00018f77f2f0320c557190d7a144bd…",1,"""e5f2d52b802189ee658865ca93d83a…","""dd7ddc04e1b6c2c614352b383efe2d…",2017-05-03 11:05:13,239.90,19.93,"""delivered""",2017-04-26 10:53:06,2017-05-12 16:04:24,"""f6dd3ec061db4e3987629fe6b26e5c…"
"""000229ec398224ef6ca0657da4fc70…",1,"""c777355d18b72b67abbeef9df44fd0…","""5b51032eddd242adc84c38acab88f2…",2018-01-18 14:48:30,199.00,17.87,"""delivered""",2018-01-14 14:33:31,2018-01-22 13:19:16,"""6489ae5e4333f3693df5ad4372dab6…"
"""00024acbcdf0a6daa1e931b038114c…",1,"""7634da152a4610f1595efa32f14722…","""9d7a1d34a5052409006425275ba1c2…",2018-08-15 10:10:18,12.99,12.79,"""delivered""",2018-08-08 10:00:35,2018-08-14 13:32:39,"""d4eb9395c8c0431ee92fce09860c5a…"
"""00048cc3ae777c65dbb7d2a0634bc1…",1,"""ef92defde845ab8450f9d70c526ef7…","""6426d21aca402a131fc0a5d0960a3c…",2017-05-23 03:55:27,21.90,12.69,"""delivered""",2017-05-15 21:42:34,2017-05-22 13:44:35,"""816cbea969fe5b689b39cfc97a5067…"
…,…,…,…,…,…,…,…,…,…,…
"""fff6889749958e42b47a7977a4cf0e…",1,"""75f6a4f019ec1322758d53b2fee2cc…","""058cb5aeb36d7c0fcae20fc85d5e0a…",2017-10-01 22:56:15,92.00,31.60,"""delivered""",2017-09-25 22:41:26,2017-09-28 22:38:04,"""c5d108ecadaedf367e995520d36c9b…"
"""fff6b8ca971f8e3ec822e99d0f2d3d…",1,"""9afaad66aca8b0c79e4f084a89c9c9…","""42bde9fef835393bb8a8849cb6b7f2…",2017-09-21 12:04:29,199.00,16.83,"""delivered""",2017-09-15 11:44:46,2017-09-21 19:34:19,"""2aec499f94f5e82786629a84b816ff…"
"""fff8287bbae429a99bb7e8c21d151c…",1,"""bee2e070c39f3dd2f6883a17a5f0da…","""4e922959ae960d389249c378d1c939…",2018-03-27 12:29:22,180.00,48.14,"""delivered""",2018-03-17 12:11:45,2018-04-07 10:07:48,"""6c1e92a209dbf868706caa83109094…"


Order purchase timestamps range from 2016-09-04 21:15:19 to 2018-09-03 09:06:57 (728 days)
Total price of all order items: 13,591,644
Total freight value of all order items: 2,251,910
Total revenue (price + freight): 15,843,553
